# UKBiobank metadata extractor

In [2]:
import os
import json

from lib.LORISIntrument import LORISIntrument
from lib.LORISField import LORISField

from lib.UKBiobankSchemas import UKBiobankSchemas

def pretty_print(string):
    print(json.dumps(string, indent=2, sort_keys=True))
    
'''
  This helps translate the UKBiobank metadata into LORIS terms
'''
class LORISHelper:
    def __init__(self, schema):
        self.__schema = schema
        
    def generate_LINST_files(self, path = './instruments'):
        if not os.path.exists(path):
            os.mkdir(path)
            
        for obj in self.__schema.categories_with_fields:
            category = obj.get('category')
            fields = obj.get('fields')
            instrument = LORISIntrument(category, fields).as_LINST()
            if instrument is not None:
                filename = instrument[0].split('{@}')[1].strip() + '.linst'
                file = open(os.path.join('instruments', filename), 'w')
                for row in instrument:
                    file.write(row)
        
    def print_tests_batteries(self):
        insert_statement = 'INSERT INTO test_battery (Test_name, AgeMinDays, AgeMaxDays, Stage, SubprojectID, Visit_label) VALUES '
        values = []
        for obj in self.__schema.categories_with_fields:
            category = obj.get('category')
            fields = obj.get('fields')
            instrument = LORISIntrument(category, fields)
            table_name = instrument.table_name
            
            visits_set = set()
            for visits in instrument.instances:
                for v in visits:
                    visits_set.add(v)
            
            for visit_label in visits_set:
                values.append('({}, 0, 2147483647, "Visit", NULL, {})'.format(table_name, visit_label))
        
            if values:
                print(insert_statement)
                print(',\n'.join(values))
                print(';')
                values = []
     
    def print_field_instances(self, field_id = ''):
        assert len(field_id), __class__.__name__ + ': field_id is required'
        try:
            raw_field = self.__schema.get_field(field_id)
        except KeyError:
            return 'KeyError: field not found'
        response = {
            "subproject": raw_field.get('instance_id'),
            "visit": raw_field.get('instance', {}).get('visits', [])
        }
        pretty_print(response)
     
    
    def install_instruments(self, baseurl, path = './instruments'):
        import getpass
        import requests
        import warnings
        
        warnings.simplefilter('ignore') # Because I am using unverified ssl certificates
        
        credentials = {
            'username': input('username: '), 
            'password': getpass.getpass('password: ')
        }
        
        token = json.loads(requests.post(
            url = baseurl + '/api/v0.0.3-dev/login',
            verify = False,
            json = credentials,
        ).content.decode('ascii')).get('token', None)
        
        for filename in os.listdir(path):
            files = {'install_file': (
                filename, 
                open(os.path.join(path, filename), "r"),
                'application/octet-stream'
            )}
            
            response = requests.post(
                url = baseurl + '/instrument_manager/?format=json',
                headers = {'Authorization': 'Bearer %s' % token},
                verify = False,
                data = {},
                files = files
            )
            
            if not response.ok:
                print(response)
        
    def generate_field_instrument_mapping(self):
        file = open('field_instrument_column_mapping.csv', 'w')
        file.write(','.join(['field_id', 'table_name', 'column_name', 'subproject_id']) + '\n')
        for obj in self.__schema.categories_with_fields:
            category = obj.get('category')
            fields = obj.get('fields')
            instrument = LORISIntrument(category, fields)
            table_name = instrument.table_name
            for props in fields:
                field_id = props['field_id']
                subproject_id = props['instance_id']
                field = LORISField(props)
                column_name = field.column_name
                file.write(','.join([field_id, table_name, column_name, subproject_id]) + '\n')

    def print_subproject_visit_indexes(self):
        pretty_print(self.__schema.subprojects)
        
x = LORISHelper(UKBiobankSchemas(cachedir = os.path.join(os.getcwd(), '.cache')))
x.generate_LINST_files() # Will create linst files under ./instruments/
#x.print_tests_batteries() # Will print insert statements for test_battery table            
#x.print_field_instances('110002') # Will list the visit_label of expected visits for a given field_id
#x.install_instruments(baseurl = '', path = './instruments') # Will post linst instruments to /instrument_manager 
#x.generate_field_instrument_mapping()
#x.print_subproject_visit_indexes()